# PCR report Work In Progress :)

In [ ]:
DATA_DIR = './data'

In [ ]:
from pcrep import parse_input

input = '230901_GN004308-086_20230901_112734_186.csv'
dc = parse_input.parse_inputname(input)
dc

In [ ]:
# 230901_GN004308-086.md
out_name = dc['date'] + dc['gn'] + '.md'
out_name

In [ ]:
import pandas as pd
import os

INPUT_PCR_DATA = "./example/230901_GN004308-086/230901_GN004308-086_20230901_112734_186.csv"
df = pd.read_csv(INPUT_PCR_DATA, delimiter=';')
df = df.replace(',', '.', regex=True)
CONC_NAME = 'Conc(copies/µL)'
df[CONC_NAME] = df[CONC_NAME].astype('Float64')

DIL_FINAL_FACTOR_NAME = 'final dilution factor'
DIL_SAMPLE_DESCRIPTION_NAME = 'Sample name'
DIL_TYPE_NAME = 'type'


def get_dir(path_name):
    if not os.path.isfile(path_name):
        raise Exception('Not file!')
    return os.path.split(path_name)[0]


WORK_DIR = get_dir(INPUT_PCR_DATA)

parsedc = parse_input.parse_inputname(os.path.split(INPUT_PCR_DATA)[1])
BASE_FILEPATH = os.path.join(
    WORK_DIR, '{}_{}'.format(parsedc['date'], parsedc['gn']))
display(WORK_DIR)
display(BASE_FILEPATH)

In [ ]:
INPUT_CONCENTRATION_DATA = "./example/230901_GN004308-086/230901_GN004308-086_conc.csv"
df_conc = pd.read_csv(INPUT_CONCENTRATION_DATA)

df_conc.set_index(['sample_id'], inplace=True)
df_conc

In [ ]:
FDL_NAME = "final dilution factor"
SAMPLE_NAME = "Sample"
SAMPLE_TYPE_NAME = "sample type"
SAMPLE_NUM_NAME = "Sample description 1"

df.loc[:, [FDL_NAME]] = df[SAMPLE_NUM_NAME].map(
    df_conc[DIL_FINAL_FACTOR_NAME], na_action='ignore')

df.loc[:, [SAMPLE_NAME]] = df[SAMPLE_NUM_NAME].map(
    df_conc[DIL_SAMPLE_DESCRIPTION_NAME], na_action='ignore')

df.loc[:, [SAMPLE_TYPE_NAME]] = df[SAMPLE_NUM_NAME].map(
    df_conc[DIL_TYPE_NAME], na_action='ignore')

df = df.dropna(subset=[SAMPLE_TYPE_NAME])
df

## Compute results

In [ ]:
def result_fn(conc, dil, a=20.0, b=2.0):
    """Compute results

    Parameters:
    conc : float
    dil : float
        final dilution factor of the sample
    a : float
        ddPCR Volume 20 µL
    b : float
        Sample volume in the ddPCR reaction 2 µL
    """
    return ((a * conc) * (1000.0 / b)) * dil


WELL_RESULT_NAME = 'vg/ml'
df.loc[:, [WELL_RESULT_NAME]] = df.apply(lambda x: result_fn(
    x['Conc(copies/µL)'], x['final dilution factor']), axis=1)

### Limits

In [ ]:
PLASMID_CONTROL_LIMITS_FILE = 'plasmid_control_limits.csv'
palsmid_control_limits = pd.read_csv(
    os.path.join(DATA_DIR, PLASMID_CONTROL_LIMITS_FILE))
palsmid_control_limits.set_index(['Target'], inplace=True)
palsmid_control_limits

In [ ]:
REFERENCE_CONTROL_LIMITS_FILE = 'reference_control_limits.csv'
reference_control_limits = pd.read_csv(
    os.path.join(DATA_DIR, REFERENCE_CONTROL_LIMITS_FILE))
reference_control_limits.set_index(['Target'], inplace=True)
reference_control_limits

How to access limits

In [ ]:
rcl = reference_control_limits
lmts = rcl.loc['IDT']
lmts['upper 3s action']

In [ ]:
from pcrep.check import METHOD_LIMIT_MULTIPLIER_NEGATIVE_CONTROL
METHOD_TARGET_ID = 'IDT'
METHOD_LIMIT_MULTIPLIER = 1.0e3  # conversion μl -> ml
method_limits = pd.read_csv('./data/method_limits.csv')

# method_limits['Lower [vg/μl]'] = method_limits['Lower [vg/μl]'].multiply(
#     METHOD_LIMIT_MULTIPLIER)
# method_limits.rename(columns={"Lower [vg/μl]": "Lower [vg/ml]"}, inplace=True)

# method_limits['Upper [vg/μl]'] = method_limits['Upper [vg/μl]'].multiply(
#     METHOD_LIMIT_MULTIPLIER)
# method_limits.rename(columns={"Upper [vg/μl]": "Upper [vg/ml]"}, inplace=True)

method_limits.set_index(['target_id'], inplace=True)
display(method_limits)

dc_limits = {'method': method_limits, 'reference_control': reference_control_limits,
             'plasmid_control': palsmid_control_limits}

In [ ]:
method_limits.loc['IDT']['Lower [vg/μl]']
mlmts = method_limits.loc['IDT']
mlmts['Lower [vg/μl]']

In [ ]:
targets = df['Target'].unique()

### Multiindex ????

In [ ]:
dfi = df.copy()
dfi.reset_index(inplace=True)
dfi.rename(columns={'Sample description 1': 'sample_id'}, inplace=True)
dfi.set_index(['sample_id', 'Target', 'Well'], inplace=True)
dfi.sort_index(inplace=True)
dfi.sort_index(axis=1)
dfi.drop(['Sample description 2', 'Sample description 3', 'Sample description 4',
          'TargetType', 'Supermix', 'Status', 'Experiment', 'SampleType'],
         axis=1, inplace=True)

Compute mean and standard deviation of `[vg/ml]`

In [ ]:
dfi.loc[:, ['mean [vg/ml]']
        ] = dfi.groupby(level=["sample_id", 'Target']).apply(lambda x: x['vg/ml'].mean())

dfi.loc[:, ['STDE']] = dfi.groupby(level=["sample_id", 'Target']).apply(
    lambda x: x['vg/ml'].std(ddof=0))
# dfi.head()

In [ ]:
def cv_fn(mean_vam: float, std_val: float, stype: str):
    cv = float("nan")
    # cv is not applied to negative samples
    if stype == 'nc':
        return cv

    if isinstance(mean_vam, float) and mean_vam != 0.0:
        cv = 100.0 * std_val / mean_vam
    return cv


CV_COLNAME = 'CV [%]'
dfi.loc[:, [CV_COLNAME]] = dfi.apply(lambda x: cv_fn(
    x['mean [vg/ml]'], x['STDE'], x['sample type']), axis=1)
dfi

### Method check

In [ ]:
from pcrep.check import method_check_routing

METHOD_CHECK_COLNAME = 'Conc(copies/µL)'


def method_check_fn(s):
    return method_check_routing(dc_limits['method'], s[SAMPLE_TYPE_NAME],
                                s[METHOD_CHECK_COLNAME], s.name[1])


VALUE_CHECK_NAME = 'method_check'
dfi.loc[:, [VALUE_CHECK_NAME]] = dfi.apply(
    lambda x: method_check_fn(x), axis=1)
dfi.head()

### Droplets check

In [ ]:
from pcrep.check import droplets_check

DROPLET_THRESHOLD = int(10000)
DROPLET_CHECK_COLNAME = 'Accepted Droplets'


def droplets_check_fn(s):
    return droplets_check(s[DROPLET_CHECK_COLNAME], DROPLET_THRESHOLD)


DROPLET_CHECK_NAME = 'droplet_check'
dfi.loc[:, [DROPLET_CHECK_NAME]] = dfi.apply(
    lambda x: droplets_check_fn(x), axis=1)
dfi.head()

#### Control check

In [ ]:
from pcrep.check import control_check_routing

CONTROL_CHECK_COLNAME_ORIG = 'mean [vg/ml]'


def control_check_fn(s):
    return control_check_routing(dc_limits, s[SAMPLE_TYPE_NAME],
                                 s[CONTROL_CHECK_COLNAME_ORIG], s.name[1])


CONTROL_CHECK_NAME = 'control_check'
dfi.loc[:, [CONTROL_CHECK_NAME]] = dfi.apply(
    lambda x: control_check_fn(x), axis=1)
dfi.head()

#### CV check

In [ ]:
from pcrep.check import cv_check


def cv_check_fn(cv_val: float):
    return cv_check(cv_val)


CV_CHECK_NAME = 'cv_check'
dfi.loc[:, [CV_CHECK_NAME]] = dfi.apply(
    lambda x: cv_check_fn(x[CV_COLNAME]), axis=1)
dfi

In [ ]:
dfi.to_excel(BASE_FILEPATH + '-data_analysis.xlsx')

#### Get sample...

In [ ]:
def get_sample(samnple_num, target_type):
    idx = pd.IndexSlice
    return dfi.loc[idx[samnple_num, target_type, :], :]


tmps = get_sample(2, 'IDT')
tmps

In [ ]:
import json

params_file = './data/params.json'
with open(params_file) as json_file:
    check_params = json.load(json_file)
# print(check_params)

## Export report

In [ ]:
# with open('md_intro.md', 'r') as f:
#     md_intro = f.read()
# with open('md_end.md', 'r') as f:
#     md_end = f.read()

# md_eval = final.to_markdown()

# md = md_intro + md_eval + md_end


# def save_md(file_path, md_txt):
#     try:
#         with open(file_path, 'w') as fl:
#             fl.write(md_txt)
#     except Exception as e:
#         print('Error: ' + str(e))


# MD_FILE = './example/230901_GN004308-086/230901_GN004308-086.md'
# save_md(MD_FILE, md)

# xls_path = os.path.splitext(MD_FILE)[0] + '.xlsx'
# final.to_excel(xls_path)

In [ ]:
# from pcrep import mdhandling

# with open(os.path.join(DATA_DIR, "config.json")) as json_file:
#     jd = json.load(json_file)
#     reference_doc = jd['reference_docx']
#     pdflatex_bin = jd['pdflatex_bin']
#     pandoc_bin = jd['pandoc_bin']

# mdhandling.md2docx(pandoc_bin, reference_doc, MD_FILE)
# print("Done.")